In [3]:
import pandas as pd
import numpy as np

In [2]:
df_st=pd.read_csv("students_complete.csv")
df_sc=pd.read_csv("schools_complete.csv")

nonD_st=df_st.copy()
nonD_sc=df_sc.copy()
df_sc

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [4]:
df_st.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


# District Summary

In [49]:
T_schools=df_sc["school_name"].count()
T_Students=df_st["Student ID"].count()
T_budget=df_sc["budget"].sum()
Av_math=df_st["math_score"].mean()
Av_reading=df_st["reading_score"].mean()
P_Math=df_st[df_st.math_score>=70]["math_score"].count()/T_Students
P_Reading=df_st[df_st.reading_score>=70]["reading_score"].count()/T_Students
Ov_P=(df_st[df_st.math_score>=70]["math_score"].count()+df_st[df_st.reading_score>=70]\
      ["reading_score"].count())/(2*T_Students)

print("Total # of Schools:     "+"\033[31m# ..........\033[1;m"+"\033[1;30m{:}\033[1;m".format(T_schools))
print("Total # of Students:    "+"\033[31m# ......\033[1;m"+"\033[1;30m{:,}\033[1;m".format(T_Students))
print("Total Budget:           "+"\033[31m$ ..\033[1;m"+"\033[1;30m{:,.2f}\033[1;m".format(T_budget))
print("Average Math Score:     "+"\033[31m# ..........\033[1;m"+"\033[1;30m{:,.2f}\033[1;m".format(Av_math))
print("Average Reading Score:  "+"\033[31m# ..........\033[1;m"+"\033[1;30m{:,.2f}\033[1;m".format(Av_reading))
print("% Passing Math:         "+"\033[31m% ..........\033[1;m"+"\033[1;30m{:,.2%}\033[1;m".format(P_Math))
print("% Passing Reading:      "+"\033[31m% ..........\033[1;m"+"\033[1;30m{:,.2%}\033[1;m".format(P_Reading))
print("% Overall:              "+"\033[31m% ..........\033[1;m"+"\033[1;30m{:,.2%}\033[1;m".format(Ov_P))


Total # of Schools:     # ..........15
Total # of Students:    # ......39,170
Total Budget:           $ ..24,649,428.00
Average Math Score:     # ..........78.99
Average Reading Score:  # ..........81.88
% Passing Math:         % ..........74.98%
% Passing Reading:      % ..........85.81%
% Overall:              % ..........80.39%


In [188]:
#cdlm=nonD_sc.set_index("School ID")
#cdlm["Per Student Budget"]=cdlm["budget"]/cdlm["size"]
#cdlm

In [189]:
bins=[0,69,100]
labels=["Failed","Passed"]

df_st["Reading_FP"]=pd.cut(df_st["reading_score"],bins=bins,labels=labels)
df_st["Math_FP"]=pd.cut(df_st["math_score"],bins=bins,labels=labels)
df_st.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,Reading_FP,Math_FP
0,0,Paul Bradley,M,9th,Huang High School,66,79,Failed,Passed
1,1,Victor Smith,M,12th,Huang High School,94,61,Passed,Failed
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,Passed,Failed
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,Failed,Failed
4,4,Bonnie Ray,F,9th,Huang High School,97,84,Passed,Passed


In [190]:
#gMath=df_st.groupby("Math_FP")
#gReading=df_st.groupby("Reading_FP")

In [191]:
newm_st=df_st[df_st.Math_FP=="Passed"]
newr_st=df_st[df_st.Reading_FP=="Passed"]
newm_st=(newm_st.groupby(["school_name"])["Math_FP"].count()).reset_index()
newr_st=(newr_st.groupby(["school_name"])["Reading_FP"].count()).reset_index()
#cdlm["% Passing Math"]=new_st.groupby(["school_name"])["Math_FP"].count()/cdlm["size"]
#cdlm
new_st=pd.merge(newr_st,newm_st,on="school_name")
new_st=pd.merge(new_st,df_sc,on="school_name")
new_st["% Reading Passed"]=new_st["Reading_FP"]/new_st["size"]
new_st["% Math Passed"]=new_st["Math_FP"]/new_st["size"]
new_st["Overall %"]=(new_st["Math_FP"]+new_st["Reading_FP"])/(2*new_st["size"])
new_st["Per Student Budget"]=new_st["budget"]/new_st["size"]
neww=new_st.drop(columns=["Reading_FP","Math_FP"],axis=1)

neww

,school_name,School ID,type,size,budget,% Reading Passed,% Math Passed,Overall %,Per Student Budget
0,Bailey High School,7,District,4976,3124928,0.819333,0.666801,0.743067,628.0
1,Cabrera High School,6,Charter,1858,1081356,0.970398,0.941335,0.955867,582.0
2,Figueroa High School,1,District,2949,1884411,0.807392,0.659885,0.733639,639.0
3,Ford High School,13,District,2739,1763916,0.792990,0.683096,0.738043,644.0
4,Griffin High School,4,Charter,1468,917500,0.971390,0.933924,0.952657,625.0
5,Hernandez High School,3,District,4635,3022020,0.808630,0.667530,0.738080,652.0
6,Holden High School,8,Charter,427,248087,0.962529,0.925059,0.943794,581.0
7,Huang High School,0,District,2917,1910635,0.813164,0.656839,0.735002,655.0
8,Johnson High School,12,District,4761,3094650,0.812224,0.660576,0.736400,650.0
9,Pena High School,9,Charter,962,585858,0.959459,0.945946,0.952703,609.0


In [192]:
topF=neww.sort_values("Overall %",ascending=False).head(5)
topF

,school_name,School ID,type,size,budget,% Reading Passed,% Math Passed,Overall %,Per Student Budget
1,Cabrera High School,6,Charter,1858,1081356,0.970398,0.941335,0.955867,582.0
12,Thomas High School,14,Charter,1635,1043130,0.973089,0.932722,0.952905,638.0
9,Pena High School,9,Charter,962,585858,0.959459,0.945946,0.952703,609.0
4,Griffin High School,4,Charter,1468,917500,0.971390,0.933924,0.952657,625.0
13,Wilson High School,5,Charter,2283,1319574,0.965396,0.938677,0.952037,578.0


In [193]:
tailF=neww.sort_values("Overall %",ascending=False).tail(5)
tailF

,school_name,School ID,type,size,budget,% Reading Passed,% Math Passed,Overall %,Per Student Budget
3,Ford High School,13,District,2739,1763916,0.792990,0.683096,0.738043,644.0
8,Johnson High School,12,District,4761,3094650,0.812224,0.660576,0.736400,650.0
7,Huang High School,0,District,2917,1910635,0.813164,0.656839,0.735002,655.0
2,Figueroa High School,1,District,2949,1884411,0.807392,0.659885,0.733639,639.0
10,Rodriguez High School,11,District,3999,2547363,0.802201,0.663666,0.732933,637.0


In [194]:
df_st.groupby(["grade"])["math_score"].mean().reset_index()

,grade,math_score
0,10th,78.941483
1,11th,79.083548
2,12th,78.993164
3,9th,78.935659


In [195]:
df_st.groupby(["school_name","grade"])["reading_score"].mean().reset_index()

,school_name,grade,reading_score
0,Bailey High School,10th,80.907183
1,Bailey High School,11th,80.945643
2,Bailey High School,12th,80.912451
3,Bailey High School,9th,81.303155
4,Cabrera High School,10th,84.253219
5,Cabrera High School,11th,83.788382
6,Cabrera High School,12th,84.287958
7,Cabrera High School,9th,83.676136
8,Figueroa High School,10th,81.408912
9,Figueroa High School,11th,80.640339


In [198]:
bins=[0,585,614,644,675]
labelx=["<$585","$585-$614","$615-$644","$645-$675"]
neww["$/Student"]=pd.cut(neww["Per Student Budget"],bins=bins,labels=labelx)
neww

,school_name,School ID,type,size,budget,% Reading Passed,% Math Passed,Overall %,Per Student Budget,$/Student
0,Bailey High School,7,District,4976,3124928,0.819333,0.666801,0.743067,628.0,$615-$644
1,Cabrera High School,6,Charter,1858,1081356,0.970398,0.941335,0.955867,582.0,<$585
2,Figueroa High School,1,District,2949,1884411,0.807392,0.659885,0.733639,639.0,$615-$644
3,Ford High School,13,District,2739,1763916,0.792990,0.683096,0.738043,644.0,$615-$644
4,Griffin High School,4,Charter,1468,917500,0.971390,0.933924,0.952657,625.0,$615-$644
5,Hernandez High School,3,District,4635,3022020,0.808630,0.667530,0.738080,652.0,$645-$675
6,Holden High School,8,Charter,427,248087,0.962529,0.925059,0.943794,581.0,<$585
7,Huang High School,0,District,2917,1910635,0.813164,0.656839,0.735002,655.0,$645-$675
8,Johnson High School,12,District,4761,3094650,0.812224,0.660576,0.736400,650.0,$645-$675
9,Pena High School,9,Charter,962,585858,0.959459,0.945946,0.952703,609.0,$585-$614


In [ ]:
newS=neww.groupby("$/Student")
nP=(newS["Purchase ID"].count()).reset_index().set_index("Age Groups")
nP